Wir wollen eine Reihe von .tsv-Dateien einlesen, die Ausreißer in den Count-Spalten identifizieren und die Ergebnisse visualisieren. Die Ausreißererkennung erfolgt entweder über den Interquartilsabstand (IQR) (alternativ über den Z-Score). Die Ergebnisse werden in einem neuen DataFrame gespeichert und die Ausreißer werden in den Visualisierungen hervorgehoben.

Zunächst werden alle .tsv-Dateien im Inputverzeichnis rekursiv gesucht und in einer Liste gespeichert.

In [19]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Ausgabeordner definieren
output_dir = '../cleaned_data'

# Inputverzeichnis definieren
input_dir = '../data'

# Rekursive Dateisuche nach .tsv-Dateien
def find_tsv_files(directory):
    tsv_files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tsv'):
                full_path = os.path.join(root, file)
                tsv_files_list.append(full_path)
    return tsv_files_list

# Liste der .tsv-Dateien im Inputverzeichnis
tsv_files = find_tsv_files(input_dir)
print (f"Found {len(tsv_files)} .tsv files in {input_dir}")
for file in tsv_files:
    print(file)


Found 0 .tsv files in ../data


Als nächstes definieren wir eine Funktion, die die Ausreißererkennung durchführt. Diese Funktion kann dann auf jede .tsv-Datei angewendet werden.
Im DataFrame definieren wir die erste Spalte als Gen-ID, die letzte Spalte als Symbol und die vorletzte Spalte als Entity (host/phage). Die Count-Spalten sind alle Spalten zwischen der ersten und der letzten Spalte.

Da Host und Phagen-Gene in der Regel unterschiedliche Verteilungen aufweisen, werden sie separat behandelt. Wir verwenden die Interquartilsabstand-Methode (IQR) zur Identifizierung von Ausreißern. Die Ausreißer werden in einer neuen Spalte 'is_outlier' markiert und die Informationen zu den Ausreißern werden in einer weiteren Spalte 'outlier_info' gespeichert.

Die IQR-Methode greift leider nicht optimal bei den niedrigen Counts, da sie per Definition nur Werte außerhalb des 1.5-fachen IQR als Ausreißer identifiziert. Bezüglich des lower bounds befinden wir uns somit im negativen Bereich. Daher wird zusätzlich das 0.05-Quantil für jede Spalte berechnet und alle Zeilen, deren Werte in allen Count-Spalten kleiner als das 0.05-Quantil sind, werden ebenfalls als Ausreißer markiert. Somit wollen wir sicherstellen, dass auch sehr niedrige Counts als Ausreißer identifiziert werden.

In [17]:
# Funktion zum markieren von Ausreißern
def mark_outliers(file_path, method='iqr', threshold=1.5):

    try:
        # Einlesen der TSV-Datei
        df = pd.read_csv(file_path, sep='\t')

        # Einlesen der Spaltennamen
        gene_col = df.columns[0] # erste Spalte ist immer das Gen
        symbol_col = df.columns[-1] # letzte Spalte ist immer das Symbol
        entity_col = df.columns[-2] # vorletzte Spalte ist immer die Entity

        # Count-Spalten extrahieren
        count_cols = df.columns[1:-2]

        # DataFrame für Ergebnisse initialisieren
        results_df = df.copy()
        results_df['is_outlier'] = False
        results_df['outlier_info']  = ""

        # Gruppierung nach Entity (host/phage)
        host_mask = df[entity_col] == 'host'
        phage_mask = df[entity_col] == 'phage'

        # Anzahl der Host- und Phagen-Gene
        print(f"Identifizierte Gene: {sum(host_mask)} Host-Gene, {sum(phage_mask)} Phagen-Gene")

       # Ausreißer für Host-Gene identifizieren
        if sum(host_mask) > 0:
            host_outliers = detect_outliers(df[host_mask], count_cols, method, threshold)
            for idx in host_outliers:
                results_df.loc[idx, 'is_outlier'] = True
                results_df.loc[idx, 'outlier_info'] += "Host-Outlier; "

        # Ausreißer für Phagen-Gene identifizieren
        if sum(phage_mask) > 0:
            phage_outliers = detect_outliers(df[phage_mask], count_cols, method, threshold)
            for idx in phage_outliers:
                results_df.loc[idx, 'is_outlier'] = True
                results_df.loc[idx, 'outlier_info'] += "Phagen-Outlier; "

        return results_df

    except Exception as e:
        print(f"Fehler {e}")
        return None

# Hilfsfunktion zur Ausreißererkennung
def detect_outliers(df, count_cols, method='iqr', threshold=1.5):
    outlier_indices = set()

    if method == 'iqr':
        # Interquartilsabstand-Methode
        for col in count_cols:
            Q05 = df[col].quantile(0.05)
            Q1 = df[col].quantile(0.25)
            #print(f"Q1: {Q1}")
            Q3 = df[col].quantile(0.75)
            #print(f"Q3: {Q3}")
            IQR = Q3 - Q1
            #print(f"IQR: {IQR}")
            lower_bound = Q1 - threshold * IQR
            #print(f"Lower Bound: {lower_bound}")
            upper_bound = Q3 + threshold * IQR
            #print(f"Upper Bound: {upper_bound}")
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.update(outliers)

         # Berechne das 0.05-Quantil für jede Spalte
        quantiles_05 = {col: df[col].quantile(0.05) for col in count_cols}

        # Prüfe für jede Zeile, ob alle Werte kleiner als das 0.05-Quantil sind
        for idx, row in df.iterrows():
            if all(row[col] < quantiles_05[col] for col in count_cols):
                outlier_indices.add(idx)

    elif method == 'zscore':
        # Z-Score-Methode
        for col in count_cols:
            z_scores = stats.zscore(df[col], nan_policy='omit')
            outliers = df[abs(z_scores) > threshold].index
            outlier_indices.update(outliers)

    return outlier_indices


Jetzt können wir die Funktion auf eine der .tsv-Dateien anwenden. Wir verwenden die IQR-Methode (Z-score hat nicht so gut geklappt, bleibt aber sicherheitshalber mal noch drin) mit einem Schwellenwert von 1.5. Die Ergebnisse werden in einem neuen DataFrame gespeichert und die Ausreißer werden in den Visualisierungen hervorgehoben.

Im folgenden Code-Snippet ist ein Beispielaufruf der Funktion mark_outliers zu sehen. Hier wird gerade die erste Datei in der Liste der .tsv Files verwendet. Andere Listen können aufgerufen werden, indem man den Index 0 ändert. Es gibt insgesamt 22 files.

## Datenbereinigung
(Um die Ergebnisse von User Story 2 darzustellen, wurde auf die Visualisierung aus User Story 1 verzichtet.)

Als Nächstes definieren wir eine Funktion `clean_outlier_samples`, die die in einer vorherigen Funktion (`mark_outliers`) identifizierten Ausreißer systematisch entfernt. Diese Funktion liest eine einzelne .tsv-Datei ein, markiert potenzielle Ausreißer und entfernt diese anschließend aus dem Datensatz.

Nach der Entfernung der Ausreißer wird die bereinigte Version der Datei automatisch unter einem neuen Namen (mit dem Zusatz `_cleaned.tsv`) im Ordner `../cleaned_data/` gespeichert. Dadurch bleibt der ursprüngliche Datensatz unverändert erhalten.

Diese Funktion eignet sich ideal zur Anwendung auf einzelne Datensätze und stellt sicher, dass nur hochwertige Proben für das spätere Modelltraining weiterverwendet werden.

Für den Fall, dass mehrere Datensätze gleichzeitig bereinigt werden sollen, definieren wir zusätzlich eine Batch-Funktion `batch_clean_all_tsv`. Diese durchläuft alle `.tsv`-Dateien in einem angegebenen Verzeichnis, bereinigt sie nacheinander mit der zuvor beschriebenen Methode und speichert alle bereinigten Dateien im gleichen Zielordner.

In [15]:
import glob

#Funktion zur Datenbereinigung
def clean_outlier_samples(file_path, method='iqr', threshold=1.5, output_dir="../cleaned_data"):
    # Ausreißer markieren
    cleaned_df = mark_outliers(file_path, method=method, threshold=threshold)

    if cleaned_df is None:
        print(f"Fehler beim Verarbeiten der Datei: {file_path}")
        return

    # Ausreißer entfernen
    final_df = cleaned_df[~cleaned_df["is_outlier"]].copy()
    removed = len(cleaned_df) - len(final_df)
    print(f"{removed} Ausreißer wurden entfernt aus {os.path.basename(file_path)}")

    # Datei speichern
    os.makedirs(output_dir, exist_ok=True)
    file_name = os.path.basename(file_path).replace(".tsv", "_cleaned.tsv")
    save_path = os.path.join(output_dir, file_name)
    final_df.to_csv(save_path, sep='\t', index=False)
    print(f"Gesäuberter Datensatz gespeichert unter: {save_path}")

    return final_df


def batch_clean_all_tsv(input_dir, output_dir="../cleaned_data", method='iqr', threshold=1.5):
    tsv_files = find_tsv_files(input_dir)

    if not tsv_files:
        print(f"Keine TSV-Dateien gefunden in: {input_dir}")
        return

    print(f"{len(tsv_files)} Dateien werden verarbeitet...\n")
    for file_path in tsv_files:
        clean_outlier_samples(
            file_path=file_path,
            method=method,
            threshold=threshold,
            output_dir=output_dir
        )
    print("\nAlle Dateien wurden verarbeitet.")


## Validierung der bereinigten Datensätze

Als Nächstes definieren wir zwei Funktionen zur Validierung der bereinigten Datensätze.  
Die erste Funktion `validate_cleaned_file` überprüft eine einzelne `.tsv`-Datei daraufhin, ob noch Ausreißer vorhanden sind. Dazu wird kontrolliert, ob die Spalte `is_outlier` in der Datei existiert und ob sie noch Einträge mit dem Wert `True` enthält. Ist dies der Fall, wurde die Datei nicht korrekt bereinigt. Wenn keine solchen Werte mehr vorhanden sind – oder die Spalte ganz fehlt – gilt der Datensatz als gültig.

Für eine effiziente Überprüfung mehrerer Dateien verwenden wir zusätzlich die Funktion `validate_all_cleaned_files`. Diese Funktion durchsucht ein angegebenes Verzeichnis nach allen Dateien mit dem Suffix `_cleaned.tsv` und wendet die Einzelfunktion `validate_cleaned_file` auf jede dieser Dateien an. Am Ende wird eine Zusammenfassung ausgegeben, ob alle Dateien erfolgreich validiert wurden oder ob noch Fehler bzw. Ausreißer vorhanden sind.

In [13]:
#Funktion zur Valisierung der berinigten Dateien
def validate_all_cleaned_files(directory="./cleaned_data"):

    # Alle Dateien mit dem Suffix _cleaned.tsv im Zielordner finden
    cleaned_files = find_tsv_files(directory)

    # Wenn keine Dateien gefunden wurden, abbrechen
    if not cleaned_files:
        print(f"Keine bereinigten Dateien gefunden in: {directory}")
        return

    print(f"{len(cleaned_files)} Dateien werden validiert...\n")
    all_passed = True

    # Jede Datei einzeln validieren
    for file_path in cleaned_files:
        is_valid = validate_cleaned_file(file_path)
        if not is_valid:
            all_passed = False

    # Zusammenfassung ausgeben
    if all_passed:
        print("Alle Dateien sind erfolgreich validiert.")
    else:
        print("Einige Dateien enthalten noch Ausreißer oder sind fehlerhaft.")

#Hilfsfunktion zur Valisierung einer Datei
def validate_cleaned_file(file_path):
    try:
        # TSV-Datei einlesen
        df = pd.read_csv(file_path, sep='\t')

        # Prüfen, ob die Spalte 'is_outlier' vorhanden ist
        if "is_outlier" in df.columns:
            # Wenn noch mindestens ein True-Wert vorhanden ist → ungültig
            if df["is_outlier"].any():
                print(f" Fehler: Datei enthält noch markierte Ausreißer → {os.path.basename(file_path)}")
                return False
            else:
                print(f" Datei validiert (keine Ausreißer mehr vorhanden): {os.path.basename(file_path)}")
                return True
        else:
            # Wenn die Spalte fehlt → Datei gilt als gültig
            print(f" Datei validiert (ohne 'is_outlier'-Spalte): {os.path.basename(file_path)}")
            return True

    except Exception as e:
        # Fehler beim Einlesen oder Verarbeiten
        print(f" Fehler beim Validieren der Datei {file_path}: {e}")
        return False


Als letzten Schritt der Datenbereinigung definieren wir die Funktion `finalize_cleaned_files`. Diese Funktion bereitet die bereits bereinigten und validierten `.tsv`-Dateien für die weitere Analyse oder das Modelltraining vor, indem sie Hilfsspalten entfernt, die nur für die Ausreißererkennung verwendet wurden.

Für jede Datei mit dem Suffix `_cleaned.tsv` im angegebenen Eingabeverzeichnis wird zunächst geprüft, ob sie erfolgreich validiert wurde. Nur dann wird sie weiterverarbeitet.

Im nächsten Schritt entfernt die Funktion die Spalten `is_outlier` und `outlier_info`, da diese für den eigentlichen Klassifikationsprozess nicht mehr benötigt werden. Die bereinigten und bereinigungsfreien Dateien werden anschließend mit dem Suffix `_final.tsv` im Zielordner gespeichert (z. B. `./final_data/`).

In [11]:
def finalize_cleaned_files(input_dir="./cleaned_data", output_dir="./final_data"):
    """
    Entfernt 'is_outlier'- und 'outlier_info'-Spalten aus validierten Dateien
    und speichert das Ergebnis im angegebenen Zielordner.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Alle bereinigten TSV-Dateien im Eingangsverzeichnis finden
    cleaned_files = glob.glob(os.path.join(input_dir, "*_cleaned.tsv"))

    if not cleaned_files:
        print(f"Keine bereinigten Dateien gefunden in: {input_dir}")
        return

    print(f"{len(cleaned_files)} Dateien werden finalisiert...\n")

    for file_path in cleaned_files:
        try:
            df = pd.read_csv(file_path, sep='\t')

            # Datei nur weiterverarbeiten, wenn sie gültig ist
            if not validate_cleaned_file(file_path):
                print(f"Überspringe ungültige Datei: {os.path.basename(file_path)}")
                continue

            # Entferne Spalten für Ausreißerkennzeichnung, falls vorhanden
            df.drop(columns=["is_outlier", "outlier_info"], inplace=True, errors='ignore')

            # Speichern der finalen Datei
            base_name = os.path.basename(file_path).replace("_cleaned.tsv", "_final.tsv")
            final_path = os.path.join(output_dir, base_name)
            df.to_csv(final_path, sep='\t', index=False)
            print(f"Finalisierte Datei gespeichert: {final_path}")

        except Exception as e:
            print(f"Fehler beim Finalisieren der Datei {file_path}: {e}")

    print("\nFinalisierung abgeschlossen.")


Zum Abschluss der Datenbereinigung möchten wir einen schnellen Überblick über die bereinigten Datensätze erhalten. Dazu verwenden wir die Funktion `preview_tsv_files`, die eine definierte Anzahl an Zeilen aus allen `.tsv`-Dateien in einem angegebenen Verzeichnis anzeigt.  

Diese Vorschau hilft dabei, die Struktur der Dateien zu kontrollieren und sicherzustellen, dass die Spalten `is_outlier` und `outlier_info` (sofern entfernt) nicht mehr enthalten sind.  

Zunächst rufen wir `batch_clean_all_tsv()` auf, um alle `.tsv`-Dateien im Eingabeverzeichnis zu bereinigen. Anschließend verwenden wir `preview_tsv_files`, um die ersten Zeilen der bereinigten Dateien (`_cleaned.tsv`) sowie der finalisierten Dateien (`_final.tsv`) zu betrachten.  

In [21]:
from IPython.display import display

def preview_tsv_files(directory, suffix="_cleaned.tsv", preview_rows=3):
    files = [f for f in os.listdir(directory) if f.endswith(suffix)]

    if not files:
        print(f"Keine Dateien mit dem Suffix '{suffix}' gefunden in: {directory}")
        return

    for file in files:
        print(f"{file}")
        df = pd.read_csv(os.path.join(directory, file), sep='\t')
        display(df.head(preview_rows))

input_dir = os.path.join(os.getcwd(), "testData")
output_dir = os.path.join(os.getcwd(), "cleaned_data")
final_output_dir= os.path.join(os.getcwd(), "final_data")

batch_clean_all_tsv(input_dir, output_dir)
finalize_cleaned_files(output_dir, final_output_dir)

print()
print("Previewing files with outliers removed (without 'is_outlier' and 'outlier_info' columns)")
preview_tsv_files(output_dir, "_cleaned.tsv")
print("Previewing files without outliers removed (without 'is_outlier' and 'outlier_info' columns)")
preview_tsv_files(final_output_dir, "_final.tsv")

8 Dateien werden verarbeitet...

Identifizierte Gene: 5665 Host-Gene, 54 Phagen-Gene
1863 Ausreißer wurden entfernt aus Brandao_LB_full_raw_counts.tsv
Gesäuberter Datensatz gespeichert unter: D:\jupyterPlace\cleaned_data\Brandao_LB_full_raw_counts_cleaned.tsv
Identifizierte Gene: 5665 Host-Gene, 376 Phagen-Gene
1439 Ausreißer wurden entfernt aus Ceyssens_non-directional_full_raw_counts.tsv
Gesäuberter Datensatz gespeichert unter: D:\jupyterPlace\cleaned_data\Ceyssens_non-directional_full_raw_counts_cleaned.tsv
Identifizierte Gene: 4484 Host-Gene, 60 Phagen-Gene
892 Ausreißer wurden entfernt aus Guegler_T7_minusToxIN_full_raw_counts.tsv
Gesäuberter Datensatz gespeichert unter: D:\jupyterPlace\cleaned_data\Guegler_T7_minusToxIN_full_raw_counts_cleaned.tsv
Identifizierte Gene: 3911 Host-Gene, 372 Phagen-Gene
805 Ausreißer wurden entfernt aus Leskinen_full_raw_counts.tsv
Gesäuberter Datensatz gespeichert unter: D:\jupyterPlace\cleaned_data\Leskinen_full_raw_counts_cleaned.tsv
Identifiziert

,Geneid,0_R1,0_R2,0_R3,5_R1,5_R2,5_R3,10_R1,10_R2,10_R3,15_R1,15_R2,15_R3,Entity,Symbol,is_outlier,outlier_info
0,gene-PA2589,12,13,24,4,10,119,1,2,19,0,0,203,host,gene-PA2589,False,NaN
1,gene-PA4119,14,15,14,17,21,96,9,4,10,6,3,175,host,aph,False,NaN
2,gene-PA2246,30,21,8,9,21,101,1,2,8,0,2,72,host,bkdR,False,NaN


Ceyssens_non-directional_full_raw_counts_cleaned.tsv


,Geneid,0_R1,0_R2,10_R1,10_R2,35_R1,35_R2,Entity,Symbol,is_outlier,outlier_info
0,gene-PA1715,3,22,0,0,0,1,host,pscB,False,NaN
1,gene-PA2803,7,2,0,0,0,0,host,gene-PA2803,False,NaN
2,gene-PA5220,7,7,0,0,0,0,host,gene-PA5220,False,NaN


Guegler_T7_minusToxIN_full_raw_counts_cleaned.tsv


...

,Geneid,0_R1,0_R2,0_R3,30_R1,30_R2,30_R3,45_R1,45_R2,45_R3,75_R1,75_R2,75_R3,135_R1,135_R2,135_R3,Entity,Symbol
0,gene-FTB24_03575,117,120,181,54,118,103,105,101,93,79,118,127,149,220,261,host,dpsA
1,gene-FTB24_07525,17,14,127,16,8,57,20,20,33,18,26,48,32,139,173,host,asrB
2,gene-FTB24_13070,24,13,111,17,21,46,21,26,25,22,26,40,41,145,104,host,spoIIIAE


Sprenger_VC_delta_tdh_VP882_WT_full_raw_counts_final.tsv


,Geneid,0_R1,0_R2,15_R2,15_R3,60_R1,60_R2,60_R3,120_R1,120_R2,120_R3,Entity,Symbol
0,gene-VC_RS11855,702,820,902,881,962,978,843,1112,1030,1019,host,rnc
1,gene-VC_RS08875,619,788,926,875,641,541,580,563,505,514,host,ybgC
2,gene-VC_RS06875,1202,1273,1493,1552,1202,1201,1030,1120,976,1134,host,gene-VC_RS06875


Wolfram-Schauerte_full_raw_counts_final.tsv


,Geneid,0_R1,0_R2,0_R3,1_R1,1_R2,1_R3,4_R2,4_R3,7_R1,7_R2,7_R3,20_R1,20_R2,20_R3,Entity,Symbol
0,gene-b1314,88,111,35,104,73,44,28,17,5,21,9,5,8,2,host,ycjR
1,gene-b1821,1691,3221,1202,6584,5665,3613,1001,1036,349,915,444,261,450,107,host,mntP
2,gene-b4418,2455,3266,1637,3663,2095,2670,868,1038,173,541,305,113,156,42,host,sraB
